In [1]:
import mysql.connector
mydb = mysql.connector.connect(host='localhost',user='root',password='Leela@123')
print(mydb.connection_id)

8


In [2]:
from tkinter import ttk
from tkinter import messagebox
import tkinter as tk
from datetime import datetime
import mysql.connector

DB_HOST = "localhost"
DB_USER = "root"
DB_PASSWORD = "Leela@123"
DB_NAME = "db"

def connect_to_db():
    try:
        conn = mysql.connector.connect(
            host=DB_HOST,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME
        )
        return conn
    except mysql.connector.Error as err:
        print("Error:", err)
        return None


def Add(root2):
    root2.withdraw()
    root11 = tk.Toplevel(root2)
    root11.title("Add Room")
    screen_width = root11.winfo_screenwidth()
    screen_height = root11.winfo_screenheight()
    window_width = 500
    window_height = 400
    position_x = (screen_width - window_width) // 2
    position_y = (screen_height - window_height) // 2
    root11.geometry(f"{window_width}x{window_height}+{position_x}+{position_y}")
    
    tk.Label(root11, text="Enter Room Details", font=("Arial", 16, "bold italic")).grid(row=0, column=0, columnspan=2, pady=20)

    tk.Label(root11, text="Room Number:", font=("Arial", 14)).grid(row=2, column=0, padx=10, pady=5, sticky="e")
    room_no = tk.Entry(root11, width=30)
    room_no.grid(row=2, column=1, padx=10, pady=10)
    
    tk.Label(root11, text="Room Type:", font=("Arial", 14)).grid(row=3, column=0, padx=10, pady=5, sticky="e")
    room_type = tk.Entry(root11, width=30)
    room_type.grid(row=3, column=1, padx=10, pady=5)

    tk.Button(root11, text="Save Room", 
              command=lambda: save_room(root11,  room_no, room_type), 
              font=("Times New Roman", 14, "bold italic"), fg="white", bg="green", width=15).grid(row=7, column=0, pady=50, padx=40)

    tk.Button(root11, text="Back", command=lambda: back_to_main(root11), font=("Times New Roman", 17, "bold italic"), 
          fg="red", bg="lightGray", width=15).grid(row=7, column=1, pady=50, padx=40)


def save_room(root11, room_no, room_type):
    conn = connect_to_db()
    if conn is None:
        tk.Label(root11, text="Database Connection Failed!", fg="red").pack()
        return

    try:
        cursor = conn.cursor()
        sql = """INSERT INTO db.room (room_no, room_type) 
                 VALUES (%s, %s)"""
        values = (room_no.get(), room_type.get())
        cursor.execute(sql, values)
        conn.commit()
        lav = tk.Label(root11, text="Room Added Successfully!", fg="green")
        lav.grid(row=6, column=0, columnspan=2, pady=10)
        
        room_no.delete(0, tk.END)
        room_type.delete(0, tk.END)
        
    except mysql.connector.Error as err:
        print("Error:", err)
        tk.Label(root11, text="Error found while Adding Room!", fg="red").grid(row=8, column=0, columnspan=2)
    finally:
        cursor.close()
        conn.close()  
 
def fetch_rooms():
    try:
        conn = mysql.connector.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWORD, database=DB_NAME)
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM db.tb")
        rooms = cursor.fetchall()
        conn.close()
        return rooms
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return []

def fetch_room1():
    try:
        conn = mysql.connector.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWORD, database=DB_NAME)
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM db.room")
        rooms = cursor.fetchall()
        conn.close()
        return rooms
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return []

def fetch_room2():
    try:
        conn = mysql.connector.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWORD, database=DB_NAME)
        cursor = conn.cursor()
        cursor.execute("""SELECT t1.customer_name, t1.phone_number, t2.room_no, t2.room_type, t1.check_in_time
                                FROM db.tb t1 JOIN db.room t2 ON t1.room_number = t2.room_no;""")
        rooms = cursor.fetchall()
        conn.close()
        return rooms
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return []
        
def room_tab(root2):
    root2.withdraw()
    root12 = tk.Toplevel(root2)
    root12.title("Rooms Table")
    root12.resizable(False, False)
    screen_width = root12.winfo_screenwidth()
    screen_height = root12.winfo_screenheight()
    window_width = 700
    window_height = 400
    position_x = (screen_width - window_width) // 2
    position_y = (screen_height - window_height) // 2
    root12.geometry(f"{window_width}x{window_height}+{position_x}+{position_y}")
    tk.Label(root12, text="Rooms Table", font=("Arial", 16, "bold")).pack(pady=10)
    columns = ("Room Number", "Room Type")
    tree = ttk.Treeview(root12, columns=columns, show="headings")
    for col in columns:
        tree.heading(col, text=col)
        tree.column(col, width=130, anchor="center")

    tree.pack(expand=True, fill="both", padx=10, pady=10)
    rooms = fetch_room1()
    for room in rooms:
        tree.insert("", "end", values=room)

    tk.Button(root12, text="Back", command=lambda: back_to_main(root12), font=("Times New Roman", 17, "bold italic"), 
          fg="red", bg="lightGray", width=15).pack(pady=20)

def check(root2):
    root2.withdraw()
    root13 = tk.Toplevel(root2)
    root13.title("Unavailable Rooms")
    root13.resizable(False, False)
    root13.geometry(f"{root13.winfo_screenwidth()}x{root13.winfo_screenheight()}")

    tk.Label(root13, text="You can check the unavailable rooms if you are facing any issues with checking all rooms", 
             font=("Arial", 14), wraplength=800, justify="center").pack(pady=40)

    tk.Label(root13, text="Type your Room Number:", font=("Arial", 14)).pack(pady=10)
    room_entry = tk.Entry(root13, font=("Arial", 14), width=30)
    room_entry.pack(pady=5)

    def check_availability():
        room_no = room_entry.get().strip()
        if not room_no:
            messagebox.showwarning("Input Error", "Please enter a room number.")
            return

        try:
            conn = connect_to_db()
            cursor = conn.cursor(buffered=True)
            cursor.execute("SELECT * FROM db.room WHERE room_no = %s", (room_no,))
            result = cursor.fetchone()
            if result:
                messagebox.showerror("Room Status", "You can't assign this Room.")
            else:
                messagebox.showinfo("Room Status", f"Room {room_no} is Available.")
        except mysql.connector.Error as err:
            messagebox.showerror("Database Error", f"Error checking room: {err}")
        finally:
            if conn:
                cursor.close()
                conn.close()
            room_no.delete(0, tk.END)

    tk.Button(root13, text="Check Availability", font=("Arial", 14, "bold"), bg="lightblue",
              command=check_availability).pack(pady=20)

    tk.Button(root13, text="Back", command=lambda: back_to_main(root13), font=("Times New Roman", 17, "bold italic"), 
          fg="red", bg="lightGray", width=15).pack(pady=20)

def room():
    root.withdraw()
    root2 = tk.Toplevel(root)
    root2.title("Room Window")
    root2.resizable(False, False)
    root2.geometry(f"{root2.winfo_screenwidth()}x{root2.winfo_screenheight()}")
    tk.Label(root2, text="Room", font=("Arial", 16)).pack(pady=20)

    tk.Button(root2, text="Add a Room", command=lambda: Add(root2), font=("Times New Roman", 17, "bold italic"), 
          fg="red", bg="lightGray", width=15).pack(padx=70, side=tk.LEFT)
    tk.Button(root2, text="Rooms Table", command=lambda: room_tab(root2), font=("Times New Roman", 17, "bold italic"), 
          fg="red", bg="lightGray", width=15).pack(padx=70, side=tk.LEFT)
    tk.Button(root2, text="Unavailable Rooms", command=lambda: check(root2), font=("Times New Roman", 17, "bold italic"), 
          fg="red", bg="lightGray", width=15).pack(padx=80, side=tk.LEFT)

    tk.Button(root2, text="Back", command=lambda: back_to_main(root2), font=("Times New Roman", 17, "bold italic"), 
          fg="red", bg="lightGray", width=15).pack(padx=50, side=tk.LEFT)


def cust():
    root.withdraw()
    root3 = tk.Toplevel(root)
    root3.title("Customers Window")
    root3.geometry("900x600")
    tk.Label(root3, text="Customer Details", font=("Arial", 16, "bold")).pack(pady=20)
    columns = ("Customer Name", "Phone Number", "Room Number", "Room Type", "Check-in Time")
    tree = ttk.Treeview(root3, columns=columns, show="headings")
    
    for col in columns:
        tree.heading(col, text=col)
        tree.column(col, width=150, anchor="center")
    tree.pack(expand=True, fill="both", padx=20, pady=10)

    rooms = fetch_room2()
    for room in rooms:
        customer_name = room[0]
        phone_number = room[1]
        room_number = room[2]
        room_type = room[3]
        check_in_time = room[4]
        tree.insert("", "end", values=(customer_name, phone_number, room_number, room_type, check_in_time))
    tk.Button(root3, text="Back", command=lambda: back_to_main(root3), font=("Times New Roman", 17, "bold italic"), 
          fg="red", bg="lightGray", width=15).pack(pady=20)


def signin():
    root.withdraw()
    root4 = tk.Toplevel(root)
    root4.title("Check In Details")
    root4.geometry("500x500")

    tk.Label(root4, text="Enter Check-In Details", font=("Arial", 16, "bold")).pack(pady=20)

    tk.Label(root4, text="Customer Name:", font=("Arial", 12)).pack()
    cname_entry = tk.Entry(root4, width=30)
    cname_entry.pack(pady=5)

    tk.Label(root4, text="Room Number:", font=("Arial", 12)).pack()
    room_entry = tk.Entry(root4, width=30)
    room_entry.pack(pady=5)

    tk.Label(root4, text="Phone Number:", font=("Arial", 12)).pack()
    phone_entry = tk.Entry(root4, width=30)
    phone_entry.pack(pady=5)

    tk.Label(root4, text="Check-In Time:", font=("Arial", 12)).pack()
    checkin_time = tk.Entry(root4, width=30)
    checkin_time.insert(0, datetime.now().strftime("%Y-%m-%d %H:%M"))
    checkin_time.pack(pady=5)

    def save_checkin():
        conn = connect_to_db()
        if conn is None:
            messagebox.showerror("Error", "Database connection failed.")
            return

        try:
            cursor = conn.cursor()
            cursor.execute("INSERT INTO db.tb (customer_name, room_number, phone_number, check_in_time) VALUES (%s, %s, %s, %s)", 
                           (cname_entry.get(), room_entry.get(), phone_entry.get(), checkin_time.get()))
            conn.commit()
            messagebox.showinfo("Success", "Check-In details saved!")

            cname_entry.delete(0, tk.END)
            room_entry.delete(0, tk.END)
            phone_entry.delete(0, tk.END)
            checkin_time.delete(0, tk.END)
            checkin_time.insert(0, datetime.now().strftime("%Y-%m-%d %H:%M"))
        except mysql.connector.Error as err:
            messagebox.showerror("Error", f"Failed to save data: {err}")
        finally:
            cursor.close()
            conn.close()

    tk.Button(root4, text="Save", command=save_checkin, font=("Times New Roman", 12, "bold"), bg="green", fg="white").pack(pady=15)

    tk.Button(root4, text="Back", command=lambda: back_to_main(root4), font=("Times New Roman", 17, "bold italic"), 
          fg="red", bg="lightGray", width=15).pack(pady=20)

def signout():
    root.withdraw()
    del_win = tk.Toplevel(root)
    del_win.title("Customer Checkout")
    del_win.geometry("400x400")

    tk.Label(del_win, text="Customer Checkout", font=("Arial", 16, "bold")).pack(pady=20)
    tk.Label(del_win, text="Enter Room Number:", font=("Arial", 12)).pack()

    room_no_entry = tk.Entry(del_win, width=30)
    room_no_entry.pack(pady=10)

    def delete_customer():
        room_no = room_no_entry.get()
        if not room_no:
            messagebox.showwarning("Input Error", "Please enter a room number.")
            return

        conn = connect_to_db()
        if conn is None:
            messagebox.showerror("Error", "Database connection failed.")
            return

        try:
            cursor = conn.cursor()
            cursor.execute("DELETE FROM db.tb WHERE room_number = %s", (room_no,))
            cursor.execute("DELETE FROM db.room WHERE room_no = %s", (room_no,))
            conn.commit()
            if cursor.rowcount > 0:
                messagebox.showinfo("Success", f"Customer in room {room_no} checked out successfully.")
            else:
                messagebox.showwarning("Not Found", f"No customer found in room {room_no}.")
        except mysql.connector.Error as err:
            messagebox.showerror("Error", f"Failed to delete customer: {err}")
        finally:
            cursor.close()
            conn.close()
            room_no_entry.delete(0, tk.END)

    tk.Button(del_win, text="Checkout", command=delete_customer, bg="green", fg="white", font=("Times New Roman", 12)).pack(pady=20)
    tk.Button(del_win, text="Back", command=lambda: back_to_main(del_win), font=("Times New Roman", 17, "bold italic"), 
          fg="red", bg="lightGray", width=15).pack(pady=20)


def prof():
    root.withdraw()
    root6 = tk.Toplevel(root)
    root6.title("Profile Window")
    root6.resizable(False, False)
    root6.geometry(f"{root6.winfo_screenwidth()}x{root6.winfo_screenheight()}")

    tk.Label(root6, text="Profile View", font=("Arial", 16)).pack(pady=20)

    tk.Label(root6, text="Enter Your Admin ID to View Profile", font=("Arial", 14)).pack(pady=10)
    admin_id_entry = tk.Entry(root6, font=("Arial", 12))
    admin_id_entry.pack(pady=5)

    def fetch_profile():
        admin_id = admin_id_entry.get().strip()
        if not admin_id:
            messagebox.showwarning("Input Error", "Please enter Admin ID")
            return

        conn = connect_to_db()
        if conn is None:
            messagebox.showerror("Database Error", "Failed to connect to database.")
            return
        
        try:
            cursor = conn.cursor()
            cursor.execute("SELECT name, email, phone_number, aadhar_number, bank_account_number, bank_type, password FROM admins WHERE admin_id = %s", (admin_id,))
            row = cursor.fetchone()
            if row:
                labels = ["Name", "Email", "Phone", "Aadhar", "Account", "Bank Type", "Password"]
                for i, value in enumerate(row):
                    tk.Label(root6, text=f"{labels[i]}: {value}", font=("Arial", 12)).pack(pady=2)
            else:
                messagebox.showinfo("Not Found", "No admin found with given ID")
        finally:
            cursor.close()
            conn.close()

    tk.Button(root6, text="View Profile", command=fetch_profile, bg="blue", fg="white", font=("Arial", 12)).pack(pady=10)

    tk.Button(root6, text="Update Details", command=lambda: update_profile_details(root6), font=("Arial", 12, "bold"), bg="green", fg="white").pack(pady=10)

    tk.Button(root6, text="Back", command=lambda: back_to_main(root6),font=("Times New Roman", 13, "bold italic"), 
              fg="red", bg="lightGray", width=13).pack(pady=20)

def update_profile_details(prev_window):
    prev_window.withdraw()
    win = tk.Toplevel(prev_window)
    win.title("Update Profile Details")
    win.geometry("400x450")

    tk.Label(win, text="Update Your Details", font=("Arial", 16, "bold")).pack(pady=20)

    tk.Label(win, text="Admin ID:", font=("Arial", 12)).pack()
    admin_id_entry = tk.Entry(win, width=30)
    admin_id_entry.pack(pady=5)

    tk.Label(win, text="New Email:", font=("Arial", 12)).pack()
    email_entry = tk.Entry(win, width=30)
    email_entry.pack(pady=5)

    tk.Label(win, text="New Phone:", font=("Arial", 12)).pack()
    phone_entry = tk.Entry(win, width=30)
    phone_entry.pack(pady=5)

    tk.Label(win, text="New Password:", font=("Arial", 12)).pack()
    password_entry = tk.Entry(win, show="*", width=30)
    password_entry.pack(pady=5)

    def update_details():
        admin_id = admin_id_entry.get().strip()
        email = email_entry.get().strip()
        phone_number = phone_entry.get().strip()
        password = password_entry.get().strip()

        if not admin_id:
            messagebox.showwarning("Input Error", "Admin ID is required.")
            return

        conn = connect_to_db()
        if conn is None:
            messagebox.showerror("Error", "Database connection failed.")
            return

        try:
            cursor = conn.cursor()
            cursor.execute("""
                UPDATE db.admins SET 
                    email = %s, 
                    phone_number = %s, 
                    password = %s 
                WHERE admin_id = %s
            """, (email, phone_number, password, admin_id))
            conn.commit()
            if cursor.rowcount > 0:
                messagebox.showinfo("Success", "Profile updated successfully!")
            else:
                messagebox.showwarning("Not Found", "No matching Admin ID.")
        except mysql.connector.Error as err:
            messagebox.showerror("Error", f"Database error: {err}")
        finally:
            cursor.close()
            conn.close()

    tk.Button(win, text="Update", command=update_details, font=("Arial", 12, "bold"), bg="green", fg="white").pack(pady=20)

    tk.Button(win, text="Back", command=lambda: back_to_main(win), font=("Times New Roman", 13, "bold italic"), 
              fg="red", bg="lightGray", width=13).pack()
    

def logout():
    root.destroy()

def back_to_main(window):
    window.destroy()
    root.deiconify()

root = tk.Tk()
root.title("Main Window")
root.resizable(False, False)
root.geometry(f"{root.winfo_screenwidth()}x{root.winfo_screenheight()}")

tk.Label(root, text="Welcome to Coders Hotel.\nThis is the Application where you are going to work", 
         font=("Calibri", 20, "bold"), fg="red").pack(padx=20, pady=50)

row1 = tk.Frame(root)
row1.pack(pady=50)

tk.Button(row1, text="Rooms", command=room, font=("Times New Roman", 17, "bold italic"), 
          fg="darkgreen", bg="yellow", width=15).pack(padx=30, side=tk.LEFT)
tk.Button(row1, text="Customers Details", command=cust, font=("Times New Roman", 17, "bold italic"), 
          fg="darkgreen", bg="yellow", width=15).pack(padx=30, side=tk.LEFT)
tk.Button(row1, text="Check In Details", command=signin, font=("Times New Roman", 17, "bold italic"), 
          fg="darkgreen", bg="yellow", width=15).pack(padx=30, side=tk.LEFT)

row2 = tk.Frame(root)
row2.pack(pady=50)

tk.Button(row2, text="Check Out Details", command=signout, font=("Times New Roman", 17, "bold italic"), 
          fg="darkgreen", bg="yellow", width=15).pack(padx=30, side=tk.LEFT)
tk.Button(row2, text="Profile View", command=prof, font=("Times New Roman", 17, "bold italic"), 
          fg="darkgreen", bg="yellow", width=15).pack(padx=30, side=tk.LEFT)
tk.Button(row2, text="Log Out", command=logout, font=("Times New Roman", 17, "bold italic"), 
          fg="darkgreen", bg="yellow", width=15).pack(padx=30, side=tk.LEFT)

root.mainloop()